# Testing with MLP(Multi-Layer Perceptron) on image data only

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
import torch.optim as optim

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [21]:
full_X_train_transformed = pd.read_csv('CSV/pre-processed/full_X_train_transformed')
full_y_train = pd.read_csv('CSV/pre-processed/full_y_train')
full_y_train = full_y_train.drop(columns=['id'])
full_y_train = full_y_train.iloc[:, 0]



full_X_test_transformed = pd.read_csv('CSV/pre-processed/full_X_test_transformed')


In [22]:
# Extracting only the image data from the full_X_train_transformed DataFrame
image_data_train = full_X_train_transformed.iloc[:, -40000:]
image_data_test = full_X_test_transformed.iloc[:, -40000:]

In [23]:
print(image_data_train.shape)
print(image_data_test.shape)
print(full_y_train.shape)

(990, 40000)
(594, 40000)
(990,)


In [24]:
image_data_train.columns = range(40000)
image_data_test.columns = range(40000)

In [25]:
encoder = LabelEncoder()
y_train_encoded = encoder.fit_transform(full_y_train)

# Splitting 20% of the training data as a validation set
X_train, X_val, y_train_encoded_split, y_val_encoded_split = train_test_split(
    image_data_train, y_train_encoded, test_size=0.2, stratify=y_train_encoded, random_state=42
)

print(f"Training set size: {X_train.shape}")
print(f"Validation set size: {X_val.shape}")


Training set size: (792, 40000)
Validation set size: (198, 40000)


In [26]:
class MLP(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, 1024)
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(1024, 512)
        self.dropout2 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, output_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x


In [27]:
# Convert data to tensors
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train_encoded_split, dtype=torch.int64).to(device)
X_val_tensor = torch.tensor(X_val.values, dtype=torch.float32).to(device)
y_val_tensor = torch.tensor(y_val_encoded_split, dtype=torch.int64).to(device)

# Define the model
n_classes = full_y_train.nunique()
model = MLP(input_dim=40000, output_dim=n_classes)
model.to(device)


# Define the loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [32]:
# Training parameters
epochs = 100
batch_size = 32

# Training loop
for epoch in range(epochs):
    for i in range(0, len(X_train_tensor), batch_size):
        # Determine end index for the current batch
        end_idx = min(i + batch_size, len(X_train_tensor))
        
        # Get the mini-batch data
        inputs = X_train_tensor[i:end_idx]
        labels = y_train_tensor[i:end_idx]

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Print statistics
    print(f"Epoch [{epoch+1}/{epochs}] - Loss: {loss.item()}")

print('Finished Training')

Epoch [1/100] - Loss: 0.010041317902505398
Epoch [2/100] - Loss: 0.0011663961922749877
Epoch [3/100] - Loss: 0.0015744155971333385
Epoch [4/100] - Loss: 0.0031500961631536484
Epoch [5/100] - Loss: 0.032679639756679535
Epoch [6/100] - Loss: 0.00014401400403585285
Epoch [7/100] - Loss: 0.00020508946909103543
Epoch [8/100] - Loss: 0.00013546342961490154
Epoch [9/100] - Loss: 0.0001192311174236238
Epoch [10/100] - Loss: 0.00011548225302249193
Epoch [11/100] - Loss: 0.00011302408529445529
Epoch [12/100] - Loss: 0.000110417022369802
Epoch [13/100] - Loss: 0.00010797390132211149
Epoch [14/100] - Loss: 0.00010579888476058841
Epoch [15/100] - Loss: 0.00010373310215072706
Epoch [16/100] - Loss: 0.00010168721928494051
Epoch [17/100] - Loss: 9.969592792913318e-05
Epoch [18/100] - Loss: 9.775434591574594e-05
Epoch [19/100] - Loss: 9.585244697518647e-05
Epoch [20/100] - Loss: 9.40548416110687e-05
Epoch [21/100] - Loss: 9.225725079886615e-05
Epoch [22/100] - Loss: 9.037030395120382e-05
Epoch [23/100]

In [33]:
# Set the model to evaluation mode
model.eval()

correct_predictions = 0
total_predictions = 0

# Prevent gradient calculations
with torch.no_grad():
    for i in range(0, len(X_val_tensor), batch_size):
        # Determine end index for the current batch
        end_idx = min(i + batch_size, len(X_val_tensor))
        
        # Get the mini-batch data
        inputs = X_val_tensor[i:end_idx]
        labels = y_val_tensor[i:end_idx]

        # Forward pass
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        
        total_predictions += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

accuracy = 100 * correct_predictions / total_predictions
print(f'Accuracy on the validation set: {accuracy:.2f}%')


Accuracy on the validation set: 54.55%


54.55% accuracy on the validation set after 300 epoch training